# Récupération des données du graphe
### Quatre fichiers tabulaires **csv** : 
* **aretes.csv** : les lignes correspondent aux arêtes, comme dans le fichier original. Quelque lignes ont été supprimées (celles dont les points sortaient du cadre défini et n'avaient pas de correspondances dans le fichier points, celles dont le point de départ était égal au point d'arrivée)
* **sommets.csv** : les lignes correspondent aux sommets du graphe.
* **matrice_poids.csv**: matrice des distances, telle que définie dans le cours pour l'algorithme de Floyd-Warshall. Les lignes et les colonnes correspondent aux identifiants des sommets.
* **matrice_adjacence.csv** : matrice d'adjacence. Les lignes et les colonnes correspondent aux identifiants des sommets.
### Deux fichiers **json**, pour une importation directe dans des dictionnaires : 
* **dicsucc.json** : pour récupérer le dictionnaire des successeurs
* **dicsuccdist.json** : pour récupérer les successeurs et les distances

## Importation des bibliothèques et répertoire par défaut


In [20]:
import pandas as pd
import numpy as np
import json
import os
# Répertoire par défaut à modifier : 
os.chdir(r'C:\INFO\S2.02 chemins bayonne')

## Importation des **json**
Par défaut les clés sont importées sous forme de chaînes de caractères, la création des dictionnaires se fait en deux temps.
### Dictionnaire dicSucc

In [21]:
# Dictionnaire des successeurs : dicsucc
# Le dictionnaire dicSuccCleStr contient des clés de type str.
# Le dictionnaire dicsucc contient des clés de type int
with open("dicsucc.json", "r") as fichier:
    dicSuccCleStr = json.load(fichier)
dicSucc = {int(k): v for k, v in dicSuccCleStr.items()}
del dicSuccCleStr

# pour afficher un extrait du dictionnaire en passant par un dataframe
pd.DataFrame(list(dicSucc.items()), columns=["sommet", "liste de successeurs"]).head()


,sommet,liste de successeurs
0,255402679,"[388382241, 1828784578, 8074523115]"
1,388382241,"[255402679, 388189690, 8254499981]"
2,388189690,"[388382241, 7055043849, 388189757]"
3,255402735,"[1828784585, 388382280, 8074523115, 388314411]"
4,1828784585,"[255402735, 1828784578, 269247892]"


In [22]:
# Dictionnaire des successeurs avec distance : dicSuccDist
# Le dictionnaire dicSuccDistCleStr contient des clés de type str.
# Le dictionnaire dicSuccDist contient des clés de type int
with open("dicsuccdist.json", "r") as fichier:
    dicSuccDistCleStr = json.load(fichier)
dicSuccDist = {int(k): v for k, v in dicSuccDistCleStr.items()}
del dicSuccDistCleStr

# pour afficher un extrait du dictionnaire en passant par un dataframe
pd.DataFrame(list(dicSuccDist.items()), columns=["sommet", "liste de successeurs avec distance"]).head()

,sommet,liste de successeurs avec distance
0,255402679,"[[388382241, 14.2], [1828784578, 29.6], [80745..."
1,388382241,"[[255402679, 14.2], [388189690, 53.4], [825449..."
2,388189690,"[[388382241, 53.4], [7055043849, 31.6], [38818..."
3,255402735,"[[1828784585, 20.5], [388382280, 36.7], [80745..."
4,1828784585,"[[255402735, 20.5], [1828784578, 60.3], [26924..."


## Importation des **csv**
Import de aretes.csv, sommets.csv, matrice_poids.csv, matrice_adjacence.csv (--> dataframe)

In [23]:
dfAretes = pd.read_table('aretes.csv', sep  =';', index_col= 0)
dfAretes.head()

,parent,lstpoints,highway,nom,surface,dist,deb,fin
id_arete,,,,,,,,
23585325,23585325,"[255402679, 388382241]",living_street,Rue Adolphe Thiers,concrete:plates,14.2,255402679,388382241
23585340,23585340,"[388382241, 6256038960, 8254499980, 388189690]",pedestrian,Place Jacques Portes,concrete:plates,53.4,388382241,388189690
24778994,24778994,"[255402735, 1828784585]",pedestrian,Rue de la Monnaie,sett,20.5,255402735,1828784585
32392579,32392579,"[364416834, 7284645447, 5940751186]",primary,Avenue Henri Grenet,asphalt,22.2,364416834,5940751186
33881143,33881143,"[388189690, 7055043849]",pedestrian,Rue Albert 1er,asphalt,31.6,388189690,7055043849


In [24]:
dfSommets = pd.read_table('sommets.csv', sep  =';', index_col= 0)
dfSommets.head()

,lat,lon,nom,equipement,bus,rue,numrue,tourism,artiste,type_art,date_debut,panoramax,tourisme,nom_artiste
id_som,,,,,,,,,,,,,,
255402679,43.491879,-1.477186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388382241,43.491996,-1.477115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388189690,43.492253,-1.477673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255402735,43.491560,-1.476544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1828784585,43.491397,-1.476664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
dfMatricePoids = pd.read_csv('matrice_poids.csv', sep = ';', index_col = 0)
dfMatricePoids.head()

MemoryError: Unable to allocate 77.6 MiB for an array with shape (3189, 3189) and data type float64

In [ ]:
dfMatriceAdj = pd.read_csv('matrice_adjacence.csv', sep = ';', index_col = 0)
dfMatriceAdj.head()

## Dictionnaires de correspondance indice <--> nom de sommet
#### **correspIndSom** : pour avoir le nom du sommet à partir de l'indice

In [ ]:
correspIndSom = {}
for i in range(len(dfMatriceAdj)) :
    correspIndSom[i] = dfMatriceAdj.index[i]

#### **correspSomInd** : pour avoir l'indice à partir du nom du sommet

In [ ]:
correspSomInd = {}
for ind in dfMatriceAdj.index : 
    correspSomInd[ind] = dfMatriceAdj.index.get_loc(ind)

## Transformation des matrices en tableaux numpy ou en listes de listes
### **Matrice d'adjacence**
* **tableau Numpy (np.array)**

In [ ]:
tabMatAdj = np.array(dfMatriceAdj)
display(tabMatAdj[0:5])

* **Liste de listes**

In [ ]:
n = len(dfMatriceAdj)
lstMatAdj = [[tabMatAdj[i,j] for j in range(n)] for i in range(n)]

### **Matrice des poids**
* **tableau Numpy (np.array)**

In [ ]:
tabMatPoids = np.array(dfMatricePoids)
display(tabMatPoids[0:5])

* **Liste de listes**

In [ ]:
n = len(tabMatPoids)
lstMatPoids = [[tabMatPoids[i,j] for j in range(n)] for i in range(n)]